In [20]:
import os
import sys
from pathlib import Path
# Setting the working directory to the root of the project
project_dir = Path("C:/Users/adbou/source/repos/KFHXRelatedAi/")
os.chdir(project_dir)

from Configs.GeneralPaths import SOURCEDATA
import pandas as pd
import numpy as np

In [21]:
#load user transactions data
transactions = pd.read_excel(Path(SOURCEDATA / "Transactions_Collab.xlsx"))

In [22]:
transactions.head()

,TrxId,TrxDate,FK_BusinessUserId,PointsRedeemed,EntityName,FK_ContentId,EntityCategory
0,12429817,2023-06-04 11:46:06.050,976480,1000,Alkhalid Clinic,113923.0,Health & Beauty
1,12525687,2023-06-07 14:08:10.287,976480,10000,H&S Store,113853.0,Electronics
2,12525666,2023-06-08 11:03:32.043,976480,10000,H&S Store,113853.0,Electronics
3,12838074,2023-06-22 08:28:30.300,976921,10,Deliveroo Plus,113851.0,Food & Beverage
4,13587992,2023-07-27 14:41:13.607,976480,50,Mawqif App,113835.0,On-demand


In [23]:
# Count the number of missing values in 'FK_ContentId'
missing_values_count = transactions['FK_ContentId'].isna().sum()

print(f"Number of missing values in 'FK_ContentId': {missing_values_count}")

Number of missing values in 'FK_ContentId': 1


In [24]:
missing_rows = transactions[transactions['FK_ContentId'].isna()]

missing_rows.head()

,TrxId,TrxDate,FK_BusinessUserId,PointsRedeemed,EntityName,FK_ContentId,EntityCategory
290,16005301,2023-10-31 00:13:46.780,990848,5000,Al Oula petrol station,NaN,NaN


In [25]:
replacement_value = 115099

# Replace missing 'FK_ContentId' values with the specified value in the main dataframe
transactions.loc[transactions['FK_ContentId'].isna(), 'FK_ContentId'] = replacement_value

# Verify the changes
updated_missing_rows = transactions[transactions['FK_ContentId'] == replacement_value]
updated_missing_rows.head()

,TrxId,TrxDate,FK_BusinessUserId,PointsRedeemed,EntityName,FK_ContentId,EntityCategory
43,15966805,2023-10-30 01:36:08.077,986005,5000,Al Oula petrol station,115099.0,NaN
44,15966513,2023-10-30 01:36:21.933,984518,5000,Al Oula petrol station,115099.0,NaN
45,15966849,2023-10-30 01:41:07.643,985948,5000,Al Oula petrol station,115099.0,NaN
46,15966838,2023-10-30 01:41:25.657,985278,5000,Al Oula petrol station,115099.0,NaN
47,15966225,2023-10-30 01:48:28.113,985583,5000,Al Oula petrol station,115099.0,NaN


In [26]:
transactions['FK_ContentId'] = transactions['FK_ContentId'].astype(int)
transactions.head()

,TrxId,TrxDate,FK_BusinessUserId,PointsRedeemed,EntityName,FK_ContentId,EntityCategory
0,12429817,2023-06-04 11:46:06.050,976480,1000,Alkhalid Clinic,113923,Health & Beauty
1,12525687,2023-06-07 14:08:10.287,976480,10000,H&S Store,113853,Electronics
2,12525666,2023-06-08 11:03:32.043,976480,10000,H&S Store,113853,Electronics
3,12838074,2023-06-22 08:28:30.300,976921,10,Deliveroo Plus,113851,Food & Beverage
4,13587992,2023-07-27 14:41:13.607,976480,50,Mawqif App,113835,On-demand


In [27]:
transactions['TrxDate'] = pd.to_datetime(transactions['TrxDate'])

In [28]:
transactions['Month'] = transactions['TrxDate'].dt.month

In [29]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    

# Apply the function to create a 'Season' column
transactions['Season'] = transactions['Month'].apply(get_season)

In [30]:
transactions.head(10)

,TrxId,TrxDate,FK_BusinessUserId,PointsRedeemed,EntityName,FK_ContentId,EntityCategory,Month,Season
0,12429817,2023-06-04 11:46:06.050,976480,1000,Alkhalid Clinic,113923,Health & Beauty,6,Summer
1,12525687,2023-06-07 14:08:10.287,976480,10000,H&S Store,113853,Electronics,6,Summer
2,12525666,2023-06-08 11:03:32.043,976480,10000,H&S Store,113853,Electronics,6,Summer
3,12838074,2023-06-22 08:28:30.300,976921,10,Deliveroo Plus,113851,Food & Beverage,6,Summer
4,13587992,2023-07-27 14:41:13.607,976480,50,Mawqif App,113835,On-demand,7,Summer
5,13901731,2023-08-09 14:41:55.927,976518,5000,COFE App,113834,On-demand,8,Summer
6,13950319,2023-08-10 15:46:13.423,976921,50,Mawqif App,113835,On-demand,8,Summer
7,14235910,2023-08-22 18:40:51.507,976921,10,Deliveroo Plus,113851,Food & Beverage,8,Summer
8,14455067,2023-08-31 12:55:37.173,976921,10,Deliveroo Plus,113851,Food & Beverage,8,Summer
9,14568055,2023-09-04 09:44:15.030,976921,10,Deliveroo Plus,113851,Food & Beverage,9,Fall


In [31]:
# Aggregate the number of transactions per FK_ContentId per season
content_season_counts = transactions.groupby(['FK_ContentId', 'Season']).size().reset_index(name='TransactionCount')

# Find the most active season for each FK_ContentId
most_active_season = content_season_counts.loc[content_season_counts.groupby('FK_ContentId')['TransactionCount'].idxmax()]

# Merge with the original DataFrame to keep the FK_ContentId details
transactions = transactions.merge(most_active_season[['FK_ContentId', 'Season']], on='FK_ContentId', suffixes=('', '_MostActiveSeason'))

# Rename the column for clarity
transactions = transactions.rename(columns={'Season_MostActiveSeason': 'MostActiveSeason'})

In [32]:
transactions[['FK_ContentId', 'MostActiveSeason']].drop_duplicates().head()

,FK_ContentId,MostActiveSeason
0,113923,Summer
1,113853,Spring
3,113851,Spring
4,113835,Fall
5,113834,Spring


In [33]:
# Determine if the transaction date is a weekday or weekend
transactions['DayOfWeek'] = transactions['TrxDate'].dt.dayofweek
transactions['IsWeekend'] = transactions['DayOfWeek'] >= 5

# Count transactions for weekdays and weekends per FK_ContentId
content_activity = transactions.groupby(['FK_ContentId', 'IsWeekend']).size().unstack(fill_value=0)
content_activity.columns = ['WeekdayTransactionCount', 'WeekendTransactionCount']

# Reset index to make FK_ContentId a column again
content_activity = content_activity.reset_index()

In [34]:
# Merge with the original DataFrame to include content activity details
transactions = transactions.merge(content_activity, on='FK_ContentId', how='left')

# Optionally, drop the intermediate 'DayOfWeek' and 'IsWeekend' columns if no longer needed
transactions = transactions.drop(columns=['DayOfWeek', 'IsWeekend'])

In [35]:
# Create a new column to indicate more active period
def determine_activity_preference(row):
    if row['WeekdayTransactionCount'] > row['WeekendTransactionCount']:
        return 'Weekdays'
    elif row['WeekendTransactionCount'] > row['WeekdayTransactionCount']:
        return 'Weekends'
    else:
        return 'Equal'

# Apply the function to create a new column
transactions['ActivityPreference'] = transactions.apply(determine_activity_preference, axis=1)

In [36]:
transactions.head()

,TrxId,TrxDate,FK_BusinessUserId,PointsRedeemed,EntityName,FK_ContentId,EntityCategory,Month,Season,MostActiveSeason,WeekdayTransactionCount,WeekendTransactionCount,ActivityPreference
0,12429817,2023-06-04 11:46:06.050,976480,1000,Alkhalid Clinic,113923,Health & Beauty,6,Summer,Summer,0,1,Weekends
1,12525687,2023-06-07 14:08:10.287,976480,10000,H&S Store,113853,Electronics,6,Summer,Spring,4,2,Weekdays
2,12525666,2023-06-08 11:03:32.043,976480,10000,H&S Store,113853,Electronics,6,Summer,Spring,4,2,Weekdays
3,12838074,2023-06-22 08:28:30.300,976921,10,Deliveroo Plus,113851,Food & Beverage,6,Summer,Spring,4140,1736,Weekdays
4,13587992,2023-07-27 14:41:13.607,976480,50,Mawqif App,113835,On-demand,7,Summer,Fall,14,0,Weekdays


In [40]:
Redemption_data = pd.read_excel(Path(SOURCEDATA / "Rdepemtion_Cleaned_Deals.xlsx"))

In [41]:
Redemption_data.head()

,Unnamed: 0.1,Unnamed: 0,ContentId,Title,FK_StatusId,Deal Type,Description,Location,Points,Categories
0,0,0,113815,Off from Al Rifai,2,Voucher,Get a 15 KD discount voucher for your next onl...,https://www.google.com/maps/search/al+rifai+ku...,15000,Food and Beverage Stores
1,1,1,113816,Off from Dip n dip,2,Voucher,Get 10 KD off on your next order from Dip n di...,NaN,10000,Restaurants and Food Services
2,2,2,113817,Order from Dip n Dip and get discount,1,Discount,Order from Dip n Dip and get 5KD discount,https://www.google.com/maps/search/dipndip/@29...,5000,Restaurants and Food Services
3,3,3,113818,Get discount from Ellena Bakery,2,Discount,Get 3KD discount from Ellena Bakery,https://maps.app.goo.gl/bnZXdfUbbaGFjFiJA,3000,Food and Beverage Stores
4,4,4,113819,Off from Talabat,1,Voucher,Get 3KD discount from Talabat.,https://goo.gl/maps/5uoJEdD95W1YSpDt9,3000,Restaurants and Food Services


In [42]:
# Rename 'ContentId' in Redemption_data to match 'FK_ContentId' in transactions
Redemption_data = Redemption_data.rename(columns={'ContentId': 'FK_ContentId'})

# Merge the dataframes on 'FK_ContentId'
merged_df = pd.merge(transactions, Redemption_data[['FK_ContentId', 'Categories']], on='FK_ContentId', how='left')

In [43]:
transactions['EntityCategory'] = merged_df['Categories']
transactions.head()

,TrxId,TrxDate,FK_BusinessUserId,PointsRedeemed,EntityName,FK_ContentId,EntityCategory,Month,Season,MostActiveSeason,WeekdayTransactionCount,WeekendTransactionCount,ActivityPreference
0,12429817,2023-06-04 11:46:06.050,976480,1000,Alkhalid Clinic,113923,Health and Medical,6,Summer,Summer,0,1,Weekends
1,12525687,2023-06-07 14:08:10.287,976480,10000,H&S Store,113853,Electronics and Appliances,6,Summer,Spring,4,2,Weekdays
2,12525666,2023-06-08 11:03:32.043,976480,10000,H&S Store,113853,Electronics and Appliances,6,Summer,Spring,4,2,Weekdays
3,12838074,2023-06-22 08:28:30.300,976921,10,Deliveroo Plus,113851,Restaurants and Food Services,6,Summer,Spring,4140,1736,Weekdays
4,13587992,2023-07-27 14:41:13.607,976480,50,Mawqif App,113835,Ground Transportation,7,Summer,Fall,14,0,Weekdays


In [44]:
#transactions.to_excel(Path(SOURCEDATA / "Redemptions_Transactions_For_Profilings.xlsx") , index=False)